In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
class TwoPhaseSimplexSolver():
    def __init__(self, A, b, c):
        self.A = A
        self.b = b
        self.c = c
        self.first_pivot = True
        self.first_tableau = np.zeros((self.A.shape[0] + 1, self.A.shape[1] + self.A.shape[0] + 2))
        self.second_tableau = np.zeros((self.A.shape[0] + 1, self.A.shape[1] + self.A.shape[0] + 1))
        self.tableau = None
    
    def __create_first_phase_tableau__(self):
        m, n = self.A.shape
        self.first_tableau[1:, 1:n + 1] = self.A
        self.first_tableau[1:, n + 1:-1] = np.eye(m)
        self.first_tableau[1:, -1] = self.b
        self.first_tableau[0, 0] = 1
        self.first_tableau[1:, 0] = -1
        return self.first_tableau

    def __create_second_phase_tablaeu__(self):
        m, n = self.A.shape
        if self.__get_status__() == 0:
            return None
        self.second_tableau = np.delete(self.first_tableau, 0, axis=1)
        self.second_tableau[0, :n] = self.c
        return self.second_tableau

    def __find_pivot__(self):
        if self.first_pivot:
            return np.argmin(self.tableau[:, -1]), 0
        pivot_col = np.argmin(self.tableau[0, :-1])
        if self.tableau[0, pivot_col] >= 0:
            return None, None
        ratios = np.array([self.tableau[i, -1] / self.tableau[i, pivot_col] if self.tableau[i, pivot_col] > 0 
                           else np.inf for i in range(1, self.tableau.shape[0])])
        if np.all(ratios == np.inf):
            return None, None
        pivot_row = np.argmin(ratios) + 1
        return pivot_row, pivot_col

    def __pivot__(self, phase):
        if phase == 1:
            self.tableau = self.first_tableau
        elif phase == 2:
            self.tableau = self.second_tableau
        else:
            raise ValueError('Phase must be 1 or 2')
        pivot_row, pivot_col = self.__find_pivot__()
        pivot = self.tableau[pivot_row, pivot_col]
        self.tableau[pivot_row, :] = self.tableau[pivot_row, :] / pivot
        for i in range(self.tableau.shape[0]):
            if i != pivot_row:
                self.tableau[i, :] -= self.tableau[i, pivot_col] * self.tableau[pivot_row, :]
        if phase == 1:
            self.first_tableau = self.tableau
        elif phase == 2:
            self.second_tableau = self.tableau
        else:
            raise ValueError('Phase must be 1 or 2')
        return self.tableau

    def __get_basic__(self):
        """
        Get basic variables
        """
        basics = []
        for j in range(self.second_tableau.shape[1] - 1):
            if np.sum(self.second_tableau[:, j] == 1) == 1 and \
                np.sum(self.second_tableau[:, j] == 0) == (self.second_tableau.shape[0] - 1):
                basics.append(j)
        return basics

    def __get_non_basic__(self):
        """
        Get non-basic variables
        """
        return [i for i in range(self.tableau.shape[1] - 1) if i not in self.__get_basic__()]

    def __get_first_tableau__(self):
        return self.first_tableau
    
    def __get_second_tableau__(self):
        return self.second_tableau

    def __get_optimal_value__(self):
        """
        Get optimal value
        """
        return -self.second_tableau[0, -1]

    def __get_solution__(self, slack=False):
        solution = np.zeros(self.second_tableau.shape[1] - 1)
        for j in self.__get_basic__():
            row = np.where(self.second_tableau[:, j] == 1)[0][0]
            solution[j] = self.second_tableau[row, -1]
        return np.array(solution[:self.A.shape[1]]) if not slack else np.array(solution)  

    def __get_status__(self):
        """
        Get status
        - No solution: 0
        - Second phase only solution: 1
        - Second phase unbounded: 2
        - Second phase infinite solution: 3
        """
        non_basic_vars = self.__get_non_basic__()
        if np.all(self.first_tableau[0, :] >= 0) and np.any(self.first_tableau[0, 1:-2] > 0):
            return 0
        elif np.all(self.second_tableau[0, 0] >= 0) and np.any(self.second_tableau[0, 1:-1] > 0):
            return 2
        elif len(non_basic_vars) > np.count_nonzero(self.second_tableau[0: -1]):
            return 3
        else:
            return 1

    def __solve_first_phase__(self):
        self.__pivot__(1)
        self.first_pivot = False
        while np.any(self.first_tableau[0, :-1] < 0):
            self.first_tableau = self.__pivot__(1)
            # Break condition
            if self.__get_status__() == 0:
                break
        return self

    def __solve_second_phase__(self):
        while np.any(self.second_tableau[0, :-1] < 0):
            self.first_tableau = self.__pivot__(2)
            # Break condition
            if self.__get_status__() == 2:
                break
        return self

    def solve(self):
        self.first_tableau = self.__create_first_phase_tableau__()
        self.__solve_first_phase__()
        print(self.__get_first_tableau__())
        if self.__get_status__() == 0:
            return self
        self.second_tableau = self.__create_second_phase_tablaeu__()
        self.__solve_second_phase__()
        print(self.__get_second_tableau__())
        if self.__get_status__() == 2:
            return self
        return self

    def get_solution(self, slack=False):
        return self.__get_solution__(slack) if self.__get_status__() == 1 else None

    def get_optimal_value(self):
        if self.__get_status__() == 0:
            return None
        elif self.__get_status__() == 2:
            return -np.inf
        return self.__get_optimal_value__()
    
    def get_status(self):
        if self.__get_status__() == 0:
            return "No solution"
        elif self.__get_status__() == 1:
            return "Only solution"
        elif self.__get_status__() == 2:
            return "Unbounded"
        else:
            return "Infinite solution"

In [3]:
A = np.array([[-1, -1, -1], [2, -1, 1]])
b = np.array([-2, 3])
c = np.array([-2, 6, 0])

solver = TwoPhaseSimplexSolver(A, b, c)
solver.__create_first_phase_tableau__()
# print(solver.__get_first_tableau__())
solver.solve()

# print(solver.__get_second_tableau__())
print(solver.get_solution())
print(solver.get_optimal_value())
print(solver.get_status())

# A = np.array([[1, 1, -4], [1, 2, -3]])
# print(A)
# print(A[:, -1])
# print(np.argmin(A[:, -1]))

[[ 1.          0.          0.          0.          0.          0.
   0.        ]
 [ 1.          0.          1.          0.33333333 -0.66666667 -0.33333333
   0.33333333]
 [ 0.          1.          0.          0.66666667 -0.33333333  0.33333333
   1.66666667]]
[[ 0.          6.          1.33333333 -0.66666667  0.66666667  3.33333333]
 [ 0.          1.          0.33333333 -0.66666667 -0.33333333  0.33333333]
 [ 1.          0.          0.66666667 -0.33333333  0.33333333  1.66666667]]
None
-inf
Unbounded


In [4]:
sol = linprog(c, A_ub=A, b_ub=b, method='highs')
print(sol)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -2.0
              x: [ 1.000e+00  0.000e+00  1.000e+00]
            nit: 2
          lower:  residual: [ 1.000e+00  0.000e+00  1.000e+00]
                 marginals: [ 0.000e+00  2.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [-2.000e+00 -2.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0
